In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

df_train = pd.read_csv('/kaggle/input/ozon-hack/ml_ozon_ounterfeit_train.csv', index_col=0)
df_test = pd.read_csv('/kaggle/input/ozon-hack/ml_ozon_ounterfeit_test.csv', index_col=0)

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")
print(f"Target distribution in train:")
print(df_train['resolution'].value_counts())
print()

Train shape: (197198, 44)
Test shape: (22760, 43)
Target distribution in train:
resolution
0    184146
1     13052
Name: count, dtype: int64



In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def preprocess_df(df: pd.DataFrame):
    df = df.copy()

    # 🎯 Целевая переменная
    y = df["resolution"].values.astype(int)

    # 🆔 ItemID — только для связи с картинкой
    ids = df["ItemID"].values

    # 📌 Удаляем ненужные колонки
    drop_cols = ["id", "resolution"]  # id — тех. колонка, resolution — таргет
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])

    # 📝 Тексты + категориальные признаки как текст
    text_cols = ["description", "name_rus", "brand_name", "CommercialTypeName4"]
    for col in text_cols:
        df[col] = df[col].fillna("missing").astype(str)

    # Числовые признаки (всё остальное)
    num_cols = df.drop(columns=text_cols).columns
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols].fillna(0))

    # 📝 Тексты в отдельный массив
    texts = df[text_cols].to_dict(orient="records")
    # будет [{description: "...", name_rus: "...", brand_name: "...", CommercialTypeName4: "..."}, ...]

    # Финальный табличный набор (без текстов)
    X_tab = df.drop(columns=text_cols).values.astype(np.float32)

    return X_tab, texts, y, ids, scaler, num_cols

In [3]:
X_tab, texts, y, ids, scaler, num_cols = preprocess_df(df_train)

print("Размер табличных фичей:", X_tab.shape)
print("Размер текстов:", len(texts))
print("Размер меток:", y.shape)
print("Примеры ItemID:", ids[:5])
print("Пример текстовой записи:", texts[0])

Размер табличных фичей: (197198, 39)
Размер текстов: 197198
Размер меток: (197198,)
Примеры ItemID: [ 78312 141999  53306 202599 163725]
Пример текстовой записи: {'description': 'Мешки пылесборники для пылесоса PHILIPS, 10 шт., синтетические, многослойные, бренд: ACTRUM, арт. AK-10/10, тип оригинального мешка: HR 6947.Подходят для пылесосов:PHILIPS: HR6955, HR6947, HR6888, HR6844 TRIATHLON, HR6843 TRIATHLON, HR6842 TRIATHLON, HR6841 TRIATHLON, HR6840 TRIATHLON, HR6839 TRIATHLON, HR6838 TRIATHLON, HR6837 TRIATHLON, HR6836 TRIATHLON, HR6835 TRIATHLON, HR6834 TRIATHLON, HR6833 TRIATHLON, HR6832 TRIATHLON, HR6831 TRIATHLON, HR6830 TRIATHLON, HR6829 TRIATHLON, HR6828 TRIATHLON, HR6827 TRIATHLON, HR6826 TRIATHLON, HR6825 TRIATHLON, HR6824 TRIATHLON, HR6823 TRIATHLON, HR6822 TRIATHLON, HR6821 TRIATHLON, HR6820 TRIATHLON, HR6819 TRIATHLON, HR6818 TRIATHLON, HR6817 TRIATHLON, HR6816 TRIATHLON, HR6815 TRIATHLON, HR6814 - HR6845 TRIATHLON, FC6844 TRIATHLON, FC6843 TRIATHLON, FC6842 TRIATHLON, FC6

In [4]:
import pandas as pd
import numpy as np
import joblib  # для сохранения объектов

numeric_cols = [f"num_{i}" for i in range(X_tab.shape[1])]

# Создаём DataFrame с табличными признаками
df_save = pd.DataFrame(X_tab, columns=numeric_cols)
df_save['ItemID'] = ids
df_save['resolution'] = y

# Добавляем текстовые колонки обратно для датасета
df_save['description'] = [t['description'] for t in texts]
df_save['name_rus'] = [t['name_rus'] for t in texts]
df_save['brand_name'] = [t['brand_name'] for t in texts]
df_save['CommercialTypeName4'] = [t['CommercialTypeName4'] for t in texts]

# Сохраняем в CSV
df_save.to_csv('train_processed.csv', index=False)

In [5]:
df_save

,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,...,num_35,num_36,num_37,num_38,ItemID,resolution,description,name_rus,brand_name,CommercialTypeName4
0,0.811289,2.293688,1.246943,0.880158,0.916662,0.031148,0.535488,-0.065227,-0.452319,-0.234959,...,-0.397470,2.358405,-0.505716,-0.126815,78312,0,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...","Мешки для пылесоса PHILIPS TRIATLON, синтетиче...",ACTRUM,Пылесборник
1,-0.072181,-0.084648,-0.088695,-0.097752,-0.102486,-0.098309,-0.114366,-0.065227,-0.614429,2.056612,...,-0.397470,2.151743,0.492936,-0.057673,141999,0,Защитная силиконовая крышка обьектива GoPro He...,Защитная крышка Redline на экшн-камеру GoPro (...,Red Line,Крышка для объектива
2,-0.072181,-0.084648,0.245214,-0.097752,-0.070638,-0.098309,-0.114366,-0.065227,-0.977287,3.896947,...,-0.397470,2.081518,-0.897826,-0.024874,53306,0,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Медиатор для гитары Acura GP-PB6,Talwar Brothers,Аксессуар для музыкального инструмента
3,-0.072181,-0.084648,-0.088695,-0.097752,-0.102486,-0.098309,-0.114366,-0.065227,0.627059,0.328349,...,-0.389294,2.021325,1.443182,-0.349756,202599,0,"Игра Sonic Frontiers для PlayStation 5, русски...","Игра Sonic Frontiers для PlayStation 5, русски...",missing,Видеоигра
4,0.075064,-0.084648,-0.088695,-0.097752,-0.102486,-0.098309,-0.114366,-0.065227,0.405946,0.452217,...,-0.389294,2.021325,0.833613,-0.349756,163725,0,Disney Classic Games: Aladdin and The Lion Kin...,"Игра Aladdin and Lion King (PlayStation 4, анг...",missing,Видеоигра
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0.516799,0.509936,-0.088695,1.858069,1.808416,3.181271,3.026596,5.016572,1.633823,0.974235,...,-0.397470,-0.135586,-0.501765,-0.035068,78564,0,Парогенератор Tefal Express Vision SV8152E0 об...,Парогенератор Tefal Express Vision SV8152E0 с ...,Tefal,Паровая станция
197194,-0.072181,-0.084648,-0.088695,0.228218,0.884814,1.325719,2.701669,6.710504,2.680858,-0.677347,...,-0.397470,0.209520,0.578082,-0.125929,147429,0,Выпрямитель для волос Dyson Airstrait<br/><br/...,Выпрямитель Dyson Airstrait HT01 Nickel / Сopper,Dyson,Выпрямитель для волос Dyson
197195,0.075064,0.509936,0.245214,-0.097752,0.757420,0.333215,1.510270,-0.065227,1.573098,0.596731,...,-0.397470,-0.304126,-1.542143,0.746332,12216,0,Настольная компактная посудомоечная машина Wei...,Weissgauff Посудомоечная машина настольная ком...,Weissgauff,Посудомоечная машина компактная
197196,8.026288,17.752867,8.926865,6.747622,2.190597,8.877385,3.351523,0.499417,-0.219011,-0.243807,...,-0.397470,-0.781656,-1.327428,1.466124,25909,1,Беспроводные наушники Redmi Buds 4 Lite Молоде...,Наушники Redmi Buds 4 Lite Black,Xiaomi,Наушники TWS


In [6]:
# preprocess_test.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from typing import List, Optional, Tuple

def preprocess_test(
    df: pd.DataFrame,
    scaler: Optional[StandardScaler] = None,
    num_cols_train: Optional[List[str]] = None,
) -> Tuple[np.ndarray, list, np.ndarray, np.ndarray, List[str]]:
    """
    Возвращает:
      X_tab        — np.float32 [N, F]
      texts        — list[dict]  [{'description', 'name_rus','brand_name','CommercialTypeName4'}, ...]
      ids_submit   — np.ndarray  id из теста (для сабмита)
      ids_item     — np.ndarray  ItemID (для картинок)
      num_cols_out — список числовых колонок, в каком порядке они ушли в X_tab
    """
    df = df.copy()

    # --- ID для сабмита ---
    if "id" in df.columns:
        ids_submit = df["id"].values
    else:
        # если читали с index_col=0 (как в примере), индекс = id
        ids_submit = df.index.values

    # --- ItemID для матчингa картинок ---
    if "ItemID" not in df.columns:
        raise ValueError("В тесте нет колонки 'ItemID'. Она нужна для поиска изображения.")
    ids_item = df["ItemID"].values

    # --- удаляем служебные колонки (таргета тут нет, но на всякий случай) ---
    drop_cols = [c for c in ["id", "resolution"] if c in df.columns]
    df = df.drop(columns=drop_cols)

    # --- тексты + «категории как текст» ---
    text_cols = ["description", "name_rus", "brand_name", "CommercialTypeName4"]
    for c in text_cols:
        df[c] = df[c].fillna("missing").astype(str)

    # --- числовые ---
    # если нам дали эталонный список колонок из train — используем его (правильный путь)
    if num_cols_train is not None:
        # добиваем недостающие колонки нулями
        for c in num_cols_train:
            if c not in df.columns:
                df[c] = 0.0
        # если в тесте есть лишние числа — игнорируем их
        num_cols = list(num_cols_train)
        # порядок фиксируем ровно как в train
        X_num = df[num_cols].fillna(0)
    else:
        # «свободный» режим: берём всё, что не текст
        num_cols = df.drop(columns=text_cols).columns.tolist()
        X_num = df[num_cols].fillna(0)

    # масштабирование (если дали scaler с train)
    if scaler is not None:
        X_num = scaler.transform(X_num)
    else:
        # без масштабирования — просто значения
        X_num = X_num.values

    # тексты в словарики
    texts = df[text_cols].to_dict(orient="records")

    # финальный массив
    X_tab = X_num.astype(np.float32)

    return X_tab, texts, ids_submit, ids_item, num_cols


In [10]:
df_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
id,,,,,,,,,,,,,,,,,,,,,
17384,NaN,Приветствую! Мы собираем классные системные бл...,"LeBefane Системный блок (AMD Ryzen 5 7500F, RA...",Настольный компьютер,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,3.0,0.000000,658.085303,768.020850,1.0,1.0,1010.0,8777,398
260316,Canon,<li>Тип товара Новый</li><li><b>Тип</b> Картри...,Картридж лазерный Canon 071H 5646C002 черный (...,"Картридж, чернила, тонер",NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,277.0,1046.284281,1259.789504,1386.227650,425.0,425.0,997.0,127929,576
10610,Devia,Электронное перо Touch S-Pen для Samsung Galax...,Стилус для Samsung Galaxy Z Fold5 5G/ SM-F946B...,Стилус,NaN,NaN,NaN,NaN,NaN,NaN,...,93.0,244.0,934.117581,1111.416588,1201.633994,9.0,9.0,812.0,5341,1946
205236,GALAXY LINE,Мультиварка GALAXY GL2643 имеет все необходимы...,"Мультиварка GALAXY GL2643 ( 900 Вт, 10 програм...","Мультиварка, скороварка",NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,61.0,1067.935416,1267.434380,1322.794836,8.0,8.0,785.0,100481,697
308655,MAUNFELD,Чайники с ретро-термометром позволяет нагреть ...,Чайник электрический MAUNFELD MGK-625MINT. Тов...,Чайник электрический,NaN,NaN,NaN,NaN,NaN,NaN,...,557.0,1476.0,1405.752984,1553.905780,1643.785320,57.0,57.0,1767.0,151937,1575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423002,NaN,Уникальный силиконовый 3D чехол для Этот гадже...,Защитные наушники Huawei Free Clip,Чехол для портативной акустики,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,206180,9051
316386,GARLYN,NaN,Пылесос вертикальный беспроводной GARLYN M-750...,Пылесос вертикальный,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,4.0,4.0,15.0,155770,1841
245410,Dell,Dell Precision T3420 — Компактная рабочая стан...,Dell Системный блок производительная рабочая с...,Настольный компьютер,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,726.445574,726.456293,726.440303,3.0,3.0,283.0,120543,9888


In [7]:
X_tab_t, texts_t, ids_submit_t, ids_item_t, num_cols_t = preprocess_test(
    df_test,
    scaler=None,              # или scaler с train
    num_cols_train=None       # или список числовых колонок из train
)

# 3) готовим датафрейм под твой Dataset (num_*, ItemID и тексты)
numeric_cols = [f"num_{i}" for i in range(X_tab_t.shape[1])]
df_save_test = pd.DataFrame(X_tab_t, columns=numeric_cols)
df_save_test["id"] = ids_submit_t        # для сабмита
df_save_test["ItemID"] = ids_item_t      # для картинок

df_save_test["description"] = [t["description"] for t in texts_t]
df_save_test["name_rus"] = [t["name_rus"] for t in texts_t]
df_save_test["brand_name"] = [t["brand_name"] for t in texts_t]
df_save_test["CommercialTypeName4"] = [t["CommercialTypeName4"] for t in texts_t]

df_save_test.to_csv("test_processed.csv", index=False)
print("✅ Saved test_processed.csv:", df_save_test.shape)

✅ Saved test_processed.csv: (22760, 45)


In [8]:
df_save_test

,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,...,num_35,num_36,num_37,num_38,id,ItemID,description,name_rus,brand_name,CommercialTypeName4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1231.452148,94.0,...,1.0,1010.0,8777.0,398.0,17384,8777,Приветствую! Мы собираем классные системные бл...,"LeBefane Системный блок (AMD Ryzen 5 7500F, RA...",missing,Настольный компьютер
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,961.077454,49.0,...,425.0,997.0,127929.0,576.0,260316,127929,<li>Тип товара Новый</li><li><b>Тип</b> Картри...,Картридж лазерный Canon 071H 5646C002 черный (...,Canon,"Картридж, чернила, тонер"
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,844.263550,48.0,...,9.0,812.0,5341.0,1946.0,10610,5341,Электронное перо Touch S-Pen для Samsung Galax...,Стилус для Samsung Galaxy Z Fold5 5G/ SM-F946B...,Devia,Стилус
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,864.502380,394.0,...,8.0,785.0,100481.0,697.0,205236,100481,Мультиварка GALAXY GL2643 имеет все необходимы...,"Мультиварка GALAXY GL2643 ( 900 Вт, 10 програм...",GALAXY LINE,"Мультиварка, скороварка"
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729.757019,47.0,...,57.0,1767.0,151937.0,1575.0,308655,151937,Чайники с ретро-термометром позволяет нагреть ...,Чайник электрический MAUNFELD MGK-625MINT. Тов...,MAUNFELD,Чайник электрический
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,881.715210,0.0,...,0.0,140.0,206180.0,9051.0,423002,206180,Уникальный силиконовый 3D чехол для Этот гадже...,Защитные наушники Huawei Free Clip,missing,Чехол для портативной акустики
22756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1025.513916,13.0,...,4.0,15.0,155770.0,1841.0,316386,155770,missing,Пылесос вертикальный беспроводной GARLYN M-750...,GARLYN,Пылесос вертикальный
22757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1164.355469,31.0,...,3.0,283.0,120543.0,9888.0,245410,120543,Dell Precision T3420 — Компактная рабочая стан...,Dell Системный блок производительная рабочая с...,Dell,Настольный компьютер
22758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,958.509033,47.0,...,239.0,287.0,214389.0,128.0,440579,214389,Кабель оптический для телевизора Samsung Q900R...,Кабель оптический для телевизора Samsung Q900R...,missing,Запчасти для телевизора


In [3]:
df_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
id,,,,,,,,,,,,,,,,,,,,,
17384,NaN,Приветствую! Мы собираем классные системные бл...,"LeBefane Системный блок (AMD Ryzen 5 7500F, RA...",Настольный компьютер,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,3.0,0.000000,658.085303,768.020850,1.0,1.0,1010.0,8777,398
260316,Canon,<li>Тип товара Новый</li><li><b>Тип</b> Картри...,Картридж лазерный Canon 071H 5646C002 черный (...,"Картридж, чернила, тонер",NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,277.0,1046.284281,1259.789504,1386.227650,425.0,425.0,997.0,127929,576
10610,Devia,Электронное перо Touch S-Pen для Samsung Galax...,Стилус для Samsung Galaxy Z Fold5 5G/ SM-F946B...,Стилус,NaN,NaN,NaN,NaN,NaN,NaN,...,93.0,244.0,934.117581,1111.416588,1201.633994,9.0,9.0,812.0,5341,1946
205236,GALAXY LINE,Мультиварка GALAXY GL2643 имеет все необходимы...,"Мультиварка GALAXY GL2643 ( 900 Вт, 10 програм...","Мультиварка, скороварка",NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,61.0,1067.935416,1267.434380,1322.794836,8.0,8.0,785.0,100481,697
308655,MAUNFELD,Чайники с ретро-термометром позволяет нагреть ...,Чайник электрический MAUNFELD MGK-625MINT. Тов...,Чайник электрический,NaN,NaN,NaN,NaN,NaN,NaN,...,557.0,1476.0,1405.752984,1553.905780,1643.785320,57.0,57.0,1767.0,151937,1575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423002,NaN,Уникальный силиконовый 3D чехол для Этот гадже...,Защитные наушники Huawei Free Clip,Чехол для портативной акустики,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,206180,9051
316386,GARLYN,NaN,Пылесос вертикальный беспроводной GARLYN M-750...,Пылесос вертикальный,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,4.0,4.0,15.0,155770,1841
245410,Dell,Dell Precision T3420 — Компактная рабочая стан...,Dell Системный блок производительная рабочая с...,Настольный компьютер,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,726.445574,726.456293,726.440303,3.0,3.0,283.0,120543,9888
